# Task 4.9 Part 1

### Import libraries and data frames

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [6]:
# import last version of orders_products_merged
path = r'C:\Users\98912\MAY 2022 Instacart Analysis'
ords_prods_merged = pd.read_pickle(os.path.join(path, '02-Data' ,'Prepared Data', 'orders_products_merged_ver3.pkl'))
ords_prods_merged.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_prder_price,spending_flag,order_frequency,frequency_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New_customer,6.367797,Low spender,20.0,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New_customer,6.367797,Low spender,20.0,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New_customer,6.367797,Low spender,20.0,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Average orders,10,New_customer,6.367797,Low spender,20.0,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Most orders,10,New_customer,6.367797,Low spender,20.0,Regular customer


In [14]:
# import the new data frame of customers info

df_customer = pd.read_csv(os.path.join(path, '02-Data' ,'Original Data', 'customers.csv'), index_col=False)
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [15]:
# check data frame dimensions
df_customer.shape

(206209, 10)

### data wrangling

In [17]:
# First rename the columns to follow similar rules for naming
df_customer = df_customer.rename(columns = {'First Name':'first_name', 'Surnam':'last_name','STATE':'state', 
                                            'n_dependants':'number_of_dependants', 'Gender': 'gender', 'Age': 'age',
                                            'fam_status': 'family_status'})
df_customer.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [20]:
# check datatype
df_customer.dtypes

user_id                  int64
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
family_status           object
income                   int64
dtype: object

In [21]:
# chanege the datatype of user_id from 'int64' to 'str'
df_customer['user_id'] = df_customer['user_id'].astype('str')

In [22]:
# duoble check data types
df_customer.dtypes

user_id                 object
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
family_status           object
income                   int64
dtype: object

#### It seems that all the columns are needed in future analysis, so we don't drop any of them. Therefore, we'll move for data consistency checks

### data consistency

In [23]:
# check the descriptive statistics for data accuracy
df_customer.describe()

,age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


#### Descriptive statistics seems to be normal and there is no data accuracy problem in numerical variables

In [25]:
# check for mixed data type in df_ords
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col + ' contains mixed data types')

first_name contains mixed data types


#### we have mixed data type in 'first_name' column. Before doing anythings about that maybe it's better to check for missing values too

In [27]:
# check for missing values
df_customer.isnull().sum()

user_id                     0
first_name              11259
last_name                   0
gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependants        0
family_status               0
income                      0
dtype: int64

#### It seems that 'first_name' column contains missing values too. As it is not a critical info in our data frame, it's better not to remove rows with null values and instead of that we substitude 'null' values with 'unknown'. it may fix the mixed data type problem as well.

In [29]:
# replace null values in 'first_name' with 'unknown'
df_customer['first_name'].fillna('unknown', inplace=True)

In [30]:
# check missing value again
df_customer.isnull().sum()

user_id                 0
first_name              0
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependants    0
family_status           0
income                  0
dtype: int64

In [31]:
# now check the mixed data type
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col + ' contains mixed data types')

#### By substituting 'null' values in 'first_name' the mixed data type problems is solved too.

In [32]:
# check for duplicated rows
df_customer.duplicated().sum()

0

#### So we don't have any duplicated rows in this data frame. As last consistency check we can observe data frequencies in some nominal variables.


In [33]:
# check data frequency in nominal variables
df_customer['user_id'].value_counts()

26711     1
67322     1
173044    1
61044     1
98344     1
         ..
146847    1
154991    1
172193    1
184326    1
80148     1
Name: user_id, Length: 206209, dtype: int64

In [34]:
df_customer['first_name'].value_counts()

unknown    11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

In [35]:
df_customer['last_name'].value_counts()

Hamilton      252
Randall       248
Lamb          243
Pennington    243
Barnett       242
             ... 
Poole         172
Bauer         166
Pearson       164
Payne         163
Jordan        162
Name: last_name, Length: 1000, dtype: int64

In [36]:
df_customer['gender'].value_counts()

Male      104067
Female    102142
Name: gender, dtype: int64

In [37]:
df_customer['state'].value_counts()

Florida                 4044
Colorado                4044
Illinois                4044
Alabama                 4044
District of Columbia    4044
Hawaii                  4044
Arizona                 4044
Connecticut             4044
California              4044
Indiana                 4044
Arkansas                4044
Alaska                  4044
Delaware                4044
Iowa                    4044
Idaho                   4044
Georgia                 4044
Wyoming                 4043
Mississippi             4043
Oklahoma                4043
Utah                    4043
New Hampshire           4043
Kentucky                4043
Maryland                4043
Rhode Island            4043
Massachusetts           4043
Michigan                4043
New Jersey              4043
Kansas                  4043
South Dakota            4043
Minnesota               4043
Tennessee               4043
New York                4043
Washington              4043
Louisiana               4043
Montana       

#### it seems a little strange that the frequency of all states are relatively similar!!! It means that the number of customers in all states are equal!!!

In [38]:
df_customer['date_joined'].value_counts()

9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: date_joined, Length: 1187, dtype: int64

In [39]:
df_customer['family_status'].value_counts()

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: family_status, dtype: int64

#### Except that case for 'state', there is not any strange thing about nominal variables data frequency. So we can move on to export the modified data fram. 

### Export customer data frame as .pkl format

In [40]:
df_customer.to_pickle(os.path.join(path, '02-Data' ,'Prepared Data', 'customers.pkl'))

### To reduce ram consumption, in this step we'll restart the kernel and reload the data frames again. 

### merge two data frames

In [41]:
# first check data type of key variables
df_customer['user_id'].dtype

dtype('O')

In [42]:
ords_prods_merged['user_id'].dtype

dtype('int64')

In [43]:
# we change the data type of key column to 'str'
ords_prods_merged['user_id'] = ords_prods_merged['user_id'].astype('str')
ords_prods_merged['user_id'].dtype

dtype('O')

In [44]:
# now we can merge two data set using 'inner_join' on 'user_id'
df_final_merged = pd.merge(df_customer, ords_prods_merged, how='inner', on='user_id')

In [45]:
df_final_merged.shape

(32404859, 33)

In [47]:
# head of the new merged data ftame
pd.options.display.max_columns = None
df_final_merged.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,family_status,income,order_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_prder_price,spending_flag,order_frequency,frequency_flag
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,2543867,5,1,9,30.0,196,2,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,8,New_customer,7.988889,Low spender,19.0,Regular customer
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,1285508,7,5,15,11.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New_customer,7.988889,Low spender,19.0,Regular customer
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,2578584,8,1,15,10.0,196,2,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,8,New_customer,7.988889,Low spender,19.0,Regular customer
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,423547,2,2,9,14.0,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,both,Mid-range product,Regularly busy,Regularly busy,Most orders,8,New_customer,7.988889,Low spender,19.0,Regular customer
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,2524893,3,3,11,30.0,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,both,Mid-range product,Regularly busy,Slowest days,Most orders,8,New_customer,7.988889,Low spender,19.0,Regular customer


In [49]:
# Export the final merged data set
df_final_merged.to_pickle(os.path.join(path, '02-Data' ,'Prepared Data', 'final_merged.pkl'))